In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from datetime import timedelta
from notebook_utils import NotebookUtilities
from pandas import DataFrame
import humanize
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re

nu = NotebookUtilities(data_folder_path=os.path.abspath('../data'))

In [3]:

# Get all CSVs into one data frame
if nu.pickle_exists('frvrs_logs_df'):
    frvrs_logs_df = nu.load_object('frvrs_logs_df')
    print(frvrs_logs_df.shape) # (842663, 112)
    # df = frvrs_logs_df.sample(4).dropna(axis='columns', how='all')
    # display(df.T)

(829277, 113)


# This is the Story of Mike_5


Mike appears in the logs right after a session-end but before a session-begin:


>SESSION_END,389471,6/12/2023 15:45,ad2137d0-d5aa-4a4e-af02-e327993c6d41,,,,,,,,,,,


>PATIENT_DEMOTED,398764,6/12/2023 15:46,ad2137d0-d5aa-4a4e-af02-e327993c6d41,100,Infinity,Mike_5 Root,"(2.4, 0.0, 2.3)","(0.0, 1.0, 0.0, 0.0)",IMMEDIATE,waver,fast,normal,normal,upset,fetal


>SESSION_START,404736,6/12/2023 15:46,ad2137d0-d5aa-4a4e-af02-e327993c6d41,,,,,,,,,,,


Mike has two injuries in need of hemorrhage control:


>PATIENT_RECORD,404737,6/12/2023 15:46,ad2137d0-d5aa-4a4e-af02-e327993c6d41,100,Infinity,Mike_5 Root,"(2.7, 0.0, 2.3)","(0.0, 1.0, 0.0, 0.0)",IMMEDIATE,waver,fast,normal,normal,upset,fetal


>INJURY_RECORD,404737,6/12/2023 15:46,ad2137d0-d5aa-4a4e-af02-e327993c6d41,R Side Puncture,Mike_5 Root,woundpack,medium,abdomen,FALSE,FALSE,"(2.6, 1.1, 2.2)",,, > ,


>INJURY_RECORD,404737,6/12/2023 15:46,ad2137d0-d5aa-4a4e-af02-e327993c6d41,R Thigh Laceration,Mike_5 Root,tourniquet,medium,rightLeg,FALSE,FALSE,"(2.5, 0.6, 2.2)",,,,,l,


Mike is asked to walk if he can:


>S_A_L_T_WALK_IF_CAN,415153,6/12/2023 15:46,ad2137d0-d5aa-4a4e-af02-e327993c6d41,"(2.7, 0.0, 2.3)",walkIfCan,Mike_5 Root,,,,,,,,,


Mike waves after he is asked to:


>S_A_L_T_WAVE_IF_CAN,426197,6/12/2023 15:46,ad2137d0-d5aa-4a4e-af02-e327993c6d41,"(2.7, 0.0, 2.3)",waveIfCan,Mike_5 Root,,,,,,,,,


>S_A_L_T_WAVED,426197,6/12/2023 15:46,ad2137d0-d5aa-4a4e-af02-e327993c6d41,"(2.7, 0.0, 2.3)",waved,Mike_5 Root,,,,,,,,,


Again, he is asked to walk:


>S_A_L_T_WALK_IF_CAN,575490,6/12/2023 15:49,ad2137d0-d5aa-4a4e-af02-e327993c6d41,"(2.7, 0.0, 2.3)",walkIfCan,Mike_5 Root,,,,,,,,,


Without ever logging a _PATIENT_ENGAGED_, one of his injuries is treated:


>TELEPORT,586184,6/12/2023 15:49,ad2137d0-d5aa-4a4e-af02-e327993c6d41,"(3.1, 0.0, 5.9)",,,,,,,,,,,


>INJURY_TREATED,595028,6/12/2023 15:49,ad2137d0-d5aa-4a4e-af02-e327993c6d41,R Thigh Laceration,Mike_5 Root,tourniquet,medium,rightLeg,TRUE,FALSE,"(3.0, 0.3, 2.2)",,,,


>TOOL_APPLIED,595028,6/12/2023 15:49,ad2137d0-d5aa-4a4e-af02-e327993c6d41,Mike_5 Root,Tourniquet,RightUpLeg (UnityEngine.GameObject),tor_ring (1) (UnityEngine.GameObject),tourniquet(Clone) (UnityEngine.GameObject),AppliedTourniquet,,,,,,


And a tag is applied:


>TAG_APPLIED,599115,6/12/2023 15:49,ad2137d0-d5aa-4a4e-af02-e327993c6d41,Mike_5 Root,red,,,,,,,,,,


>TELEPORT,601400,6/12/2023 15:49,ad2137d0-d5aa-4a4e-af02-e327993c6d41,"(3.1, 0.0, 3.9)",,,,,,,,,,,


A long time later, the system finally logs a _PATIENT_ENGAGED_ and he gets his other injury treated:


>TELEPORT,675206,6/12/2023 15:50,ad2137d0-d5aa-4a4e-af02-e327993c6d41,"(2.3, 0.0, 1.7)",,,,,,,,,,,


>PATIENT_ENGAGED,677073,6/12/2023 15:50,ad2137d0-d5aa-4a4e-af02-e327993c6d41,100,Infinity,Mike_5 Root,"(2.7, 0.0, 2.3)","(0.0, 1.0, 0.0, 0.0)",IMMEDIATE,waver,fast,normal,normal,upset,fetal


>INJURY_TREATED,684984,6/12/2023 15:50,ad2137d0-d5aa-4a4e-af02-e327993c6d41,R Side Puncture,Mike_5 Root,woundpack,medium,abdomen,TRUE,FALSE,"(3.1, 0.3, 2.5)",,,,


>TOOL_APPLIED,684984,6/12/2023 15:50,ad2137d0-d5aa-4a4e-af02-e327993c6d41,Mike_5 Root,Gauze_Pack,skinCollider_BodyCollideLOD (UnityEngine.GameObject),null,null,AppliedPackingGauze,,,,,,


# Challenge Encountered

In [4]:

search_regex = re.compile('^(tourniquet|woundpack)$')
regexed_columns = nu.get_regexed_columns(frvrs_logs_df, search_regex=search_regex, verbose=False)
regexed_df = nu.get_regexed_dataframe(frvrs_logs_df, regexed_columns, search_regex=search_regex, verbose=False)

In [5]:

import random

print(regexed_columns)
df = regexed_df.sample(min(8, regexed_df.shape[0])).dropna(axis='columns', how='all')
regexed_columns = sorted(set(regexed_columns).intersection(set(df.columns)))
# extra_columns = []
# if len(regexed_columns) < 20:
#     extra_count = 20 - len(regexed_columns)
#     extra_columns = random.sample(set(df.columns) - set(regexed_columns), extra_count)
# columns_list = list(set(extra_columns + regexed_columns).intersection(set(df.columns)))
df[regexed_columns].T.head(20)

['voice_capture_message', 'injury_record_required_procedure', 'injury_treated_required_procedure']


,98284,128,8
injury_record_required_procedure,NaN,NaN,woundpack
injury_treated_required_procedure,NaN,tourniquet,NaN
voice_capture_message,tourniquet,NaN,NaN


In [4]:

[cn for cn in frvrs_logs_df.columns if cn.endswith('_sort')]

['patient_demoted_sort', 'patient_record_sort', 'patient_engaged_sort']

In [6]:

mask_series = ~frvrs_logs_df.injury_id.isnull()
frvrs_logs_df[mask_series].action_type.unique().tolist()

['INJURY_RECORD', 'INJURY_TREATED']


## Task

We were trying to calculate time to hemorrhage control for scene defined as “time of start scene to time last hemorrhage control (last tourniquet applied)”.



    b.  Calculate time to hemorrhage control for scene defined as “time of start scene to time last hemorrhage control (last tourniquet applied)”

        Median = 3:51, SD = ±1:44, IQR = (2:41, 4:52)


## Situation

The FRVRS logger has trouble logging multiple tool applications. After the responder applied the packing and the dressing gauze, it logged another _INJURY_TREATED_, but with **injury_treated_injury_treated** == True and **injury_treated_injury_treated_with_wrong_treatment** == True. And even though he applied the right tool after that, another _INJURY_TREATED_ was never logged. So it must be combining the two _TOOL_APPLIEDs_, one with **injury_treated_injury_treated_with_wrong_treatment** == True (the _AppliedPackingGauze_) and one with **injury_treated_injury_treated** == True (the _AppliedTourniquet_).

In [8]:

# Filter out the non-aborted triage scenes in each one-per-file file
base_mask_series = (frvrs_logs_df.scene_type == 'Triage') & (frvrs_logs_df.is_scene_aborted == False) & (frvrs_logs_df.is_a_one_triage_file == True)

# Filter out the non-bleeders
procedures_list = ['tourniquet', 'woundpack']
mask_series = frvrs_logs_df.injury_record_required_procedure.isin(procedures_list) | frvrs_logs_df.injury_treated_required_procedure.isin(procedures_list)
mask_series = base_mask_series & mask_series

# Get the total scene count
total_scene_count = len(frvrs_logs_df[base_mask_series].groupby(fu.scene_groupby_columns).groups)

# Get the bleeder scene count
bleeder_scene_count = len(frvrs_logs_df[mask_series].groupby(fu.scene_groupby_columns).groups)
print(f'There are {bleeder_scene_count} scenes with bleeders in them out of a total of {total_scene_count} scenes.')

# Get the total patient count
total_patient_count = len(frvrs_logs_df[base_mask_series].groupby(fu.patient_groupby_columns).groups)

# Get the bleeder patient count
bleeder_patient_count = len(frvrs_logs_df[mask_series].groupby(fu.patient_groupby_columns).groups)
print(f'There are {bleeder_patient_count:,} bleeding patients out of a total of {total_patient_count:,} patients.')

# Get the non-engaged patient count
# mask_series = base_mask_series & frvrs_logs_df.patient_engaged_salt.isnull()

# Get the total injury count
total_injury_count = len(frvrs_logs_df[base_mask_series].groupby(fu.injury_groupby_columns).groups)

# Get the bleeding injury count
bleeding_injury_count = len(frvrs_logs_df[mask_series].groupby(fu.injury_groupby_columns).groups)
print(f'There are {bleeding_injury_count:,} bleeding injuries out of a total of {total_injury_count:,} injuries.')

There are 388 scenes with bleeders in them out of a total of 389 scenes.
There are 2,050 bleeding patients out of a total of 4,715 patients.
There are 3,487 bleeding injuries out of a total of 11,554 injuries.


In [9]:

if nu.pickle_exists('run_control_time_df'): run_control_time_df = nu.load_object('run_control_time_df')
else:
    
    # Iterate through each triage scene in each one-per-file file
    rows_list = []
    for (session_uuid, scene_index), scene_df in frvrs_logs_df[base_mask_series].groupby(fu.scene_groupby_columns):
        
        # Get the run start time
        time_start = fu.get_scene_start(scene_df)
        
        # Iterate through each patient in each triage scene
        for patient_id, patient_df in scene_df.groupby('patient_id'):
            
            # Get the engagement time
            mask_series = patient_df.action_type.isin(['PATIENT_ENGAGED'])
            patient_engaged = fu.get_scene_start(patient_df[mask_series])
    
            # Check if they are dead
            is_patient_dead = fu.is_patient_dead(patient_df)
            
            # Iterate through each injury of each patient
            for injury_id, injury_df in patient_df.groupby('injury_id'):
                row_dict = {}
                for cn in fu.injury_groupby_columns: row_dict[cn] = eval(cn)
                row_dict['scene_start'] = time_start
                row_dict['patient_engagement_time'] = patient_engaged
                row_dict['is_patient_dead'] = is_patient_dead
                
                # Is the patient bleeding?
                row_dict['is_injury_hemorrhage'] = fu.is_injury_hemorrhage(injury_df)
                row_dict['is_hemorrhage_controlled'] = fu.is_hemorrhage_controlled(injury_df)
                bleeding_mask_series = injury_df.injury_record_required_procedure.isin(procedures_list) | injury_df.injury_treated_required_procedure.isin(procedures_list)
                if (1 <= injury_df[bleeding_mask_series].shape[0] <= 2):
                    
                    # Was the bleeding treated?
                    if (injury_df[bleeding_mask_series].shape[0] == 2):
                        
                        # Get the last tourniquet applied
                        time_stop = injury_df[mask_series].elapsed_time.max()
                        row_dict['bleeding_treated_time'] = time_stop
                        
                        # Get the time elapsed
                        row_dict['run_bleeding_control_delta'] = time_stop - time_start
                        row_dict['engaged_bleeding_control_delta'] = time_stop - patient_engaged
                
                rows_list.append(row_dict)
    
    # Store the run control data frame
    run_control_time_df = DataFrame(rows_list)
    nu.store_objects(run_control_time_df=run_control_time_df, verbose=True)

In [10]:

columns_list = ['is_patient_dead', 'is_hemorrhage_controlled']
df = nu.get_minority_combinations(run_control_time_df, columns_list)
scene_groups = df.groupby(fu.scene_groupby_columns).groups
display(df.T)

,3026,6062,3039,940
session_uuid,6f1a94ad-99cb-49a6-86f8-145e6c090fae,e3c2b4c6-d8b7-4b64-8cd0-7ca0e622f9b5,705ac61e-38b7-4b89-8778-4ea58bad18a6,2270eaba-f9b5-4f96-8a5e-b69276846474
scene_index,0,1,0,0
patient_id,Lily_7 Root,Lily_4 Root,Bob_0 Root,Gary_5 Root
injury_id,L Wrist Amputation,L Side Puncture,R Shin Amputation,Asthmatic
scene_start,4539,252268,16245,5420
patient_engagement_time,110291.0,433073.0,210246.0,454042.0
is_patient_dead,True,False,True,False
is_injury_hemorrhage,True,True,True,False
is_hemorrhage_controlled,True,True,False,False
bleeding_treated_time,NaN,NaN,NaN,NaN


In [11]:

mask_series = run_control_time_df.is_patient_dead.isnull()
df = run_control_time_df[~mask_series]
scene_groups = df.groupby(fu.scene_groupby_columns).groups
df.drop_duplicates(subset=columns_list).dropna(axis='columns', how='all').T.head(20)

,0,6,12,3026
session_uuid,009b848c-ea64-4f22-bd40-711213a3d327,009b848c-ea64-4f22-bd40-711213a3d327,009b848c-ea64-4f22-bd40-711213a3d327,6f1a94ad-99cb-49a6-86f8-145e6c090fae
scene_index,0,0,0,0
patient_id,Bob_0 Root,Bob_9 Root,Lily_2 Root,Lily_7 Root
injury_id,Face Shrapnel,Ear Bleed,R Shin Amputation,L Wrist Amputation
scene_start,296666,296666,296666,4539
patient_engagement_time,704250.0,NaN,409276.0,110291.0
is_patient_dead,True,False,False,True
is_injury_hemorrhage,False,False,True,True
is_hemorrhage_controlled,False,False,True,True


In [ ]:

group_tuple = [g for g, idx_list in scene_groups.items() if (list(idx_list) == [6])][0]
mask_series = (frvrs_logs_df.session_uuid == group_tuple[0]) & (frvrs_logs_df.scene_index == group_tuple[1])# & (frvrs_logs_df.patient_id == 'Bob_0 Root')
mask_series &= ~frvrs_logs_df.action_type.isin(['VOICE_CAPTURE', 'PATIENT_DEMOTED', 'INJURY_RECORD'])
frvrs_logs_df.loc[mask_series].sort_values('elapsed_time').head(8).dropna(axis='columns', how='all').T.head(20)

In [86]:

columns_list = [cn for cn in run_control_time_df.columns if cn.endswith('_delta')]
df = run_control_time_df.groupby(fu.scene_groupby_columns).max().reset_index(drop=False)[fu.scene_groupby_columns + columns_list]
mask_series = ~df.run_bleeding_control_delta.isnull() & ~df.engaged_bleeding_control_delta.isnull()
df[mask_series]

,session_uuid,scene_index,run_bleeding_control_delta,engaged_bleeding_control_delta


In [79]:

groupby_columns = ['is_injury_hemorrhage', 'is_hemorrhage_controlled']
df = run_control_time_df.groupby(groupby_columns).size().to_frame().rename(columns={0: 'injury_count'}).reset_index(drop=False)
display(df)
total_count = df.injury_count.sum()
mask_series = (df.is_injury_hemorrhage == True)
hemorrhage_count = df[mask_series]['injury_count'].sum()
not_hemorrhage_count = df[~mask_series]['injury_count'].sum()
mask_series = (df.is_injury_hemorrhage == True) & (df.is_hemorrhage_controlled == True)
controlled_count = df[mask_series]['injury_count'].sum()
mask_series = (df.is_injury_hemorrhage == True) & (df.is_hemorrhage_controlled == False)
not_controlled_count = df[mask_series]['injury_count'].sum()
print(
    f'Of the {total_count:,} injuries, {not_hemorrhage_count:,} are not hemorrhages. Of the {hemorrhage_count:,} injuries that are,'
    f' {not_controlled_count:,} are not controlled.'
)

,is_injury_hemorrhage,is_hemorrhage_controlled,injury_count
0,False,False,3558
1,True,False,1708
2,True,True,1573


Of the 6,839 injuries, 3,558 are not hemorrhages. Of the 3,281 injuries that are, 1,708 were not controlled.


In [ ]:

# display(frvrs_logs_df.injury_record_required_procedure.unique().tolist())
scene_action_types_set = set()
patient_action_types_set = set()
injury_action_types_set = set()

# Iterate through each injury of each patient in each triage scene in each one-per-file file
for (session_uuid, scene_index, patient_id, injury_id), injury_df in gb:
    bleeding_mask_series = injury_df.injury_record_required_procedure.isin(procedures_list) | injury_df.injury_treated_required_procedure.isin(procedures_list)
    if (1 <= injury_df[bleeding_mask_series].shape[0] <= 2):
        
        # Get the entire history of this scene
        mask_series = True
        for cn in fu.scene_groupby_columns: mask_series &= (frvrs_logs_df[cn] == eval(cn))
        scene_df = frvrs_logs_df[mask_series]
        
        # Get the action types associated with the relevent action history of this scene
        mask_series = scene_df.action_type.isin(['PATIENT_RECORD', 'INJURY_RECORD', 'PATIENT_ENGAGED', 'TOOL_SELECTED', 'TOOL_APPLIED', 'INJURY_TREATED'])
        # display(scene_df[mask_series].dropna(axis='columns', how='all').T); raise
        for action_type in scene_df[mask_series].action_type.unique(): scene_action_types_set.add(action_type)
        
        # Get the entire history of this patient
        mask_series = True
        for cn in fu.patient_groupby_columns: mask_series &= (frvrs_logs_df[cn] == eval(cn))
        patient_df = frvrs_logs_df[mask_series]
        
        # Get the action types associated with the relevent action history of this patient
        mask_series = patient_df.action_type.isin(['PATIENT_RECORD', 'INJURY_RECORD', 'PATIENT_ENGAGED', 'TOOL_SELECTED', 'TOOL_APPLIED', 'INJURY_TREATED'])
        # display(patient_df[mask_series].dropna(axis='columns', how='all').T); raise
        for action_type in patient_df[mask_series].action_type.unique(): patient_action_types_set.add(action_type)
        
        # Get the action types associated with the relevent action history of this injury
        mask_series = injury_df.action_type.isin(['TOOL_SELECTED', 'PATIENT_RECORD', 'INJURY_RECORD', 'PATIENT_ENGAGED', 'TOOL_APPLIED', 'INJURY_TREATED'])
        # display(injury_df[mask_series].dropna(axis='columns', how='all').T); raise
        for action_type in injury_df[mask_series].action_type.unique(): injury_action_types_set.add(action_type)

In [46]:

# {'TOOL_SELECTED'}
scene_action_types_set - patient_action_types_set

{'TOOL_SELECTED'}

In [47]:

# {'PATIENT_RECORD', 'TOOL_APPLIED', 'PATIENT_ENGAGED', 'INJURY_TREATED'}
patient_action_types_set - injury_action_types_set

{'PATIENT_RECORD', 'TOOL_APPLIED', 'PATIENT_ENGAGED'}

In [48]:

# {'INJURY_RECORD'}
injury_action_types_set

{'INJURY_RECORD', 'INJURY_TREATED'}

In [16]:

# Get the total and bleeder scene count
base_mask_series = (frvrs_logs_df.scene_type == 'Triage') & (frvrs_logs_df.is_scene_aborted == False) & (frvrs_logs_df.is_a_one_triage_file == True)
mask_series = base_mask_series & frvrs_logs_df.injury_treated_required_procedure.isin(['tourniquet', 'woundpack'])
total_scene_count = len(frvrs_logs_df[base_mask_series].groupby(fu.scene_groupby_columns).groups)
gb = frvrs_logs_df[mask_series].groupby(fu.scene_groupby_columns)
bleeder_scene_count = len(gb.groups)

# Iterate through each one-per-file, triage bleeder scene in each file
rows_list = []
file_names_set = set()
is_first_one = True
for (session_uuid, scene_index), tourniquet_required_scene_df in gb:
    row_dict = {}
    for cn in fu.scene_groupby_columns: row_dict[cn] = eval(cn)
    
    # Get the entire history of this scene for the run start
    mask_series = True
    for cn in fu.scene_groupby_columns: mask_series &= (frvrs_logs_df[cn] == eval(cn))
    scene_df = frvrs_logs_df[mask_series]
    time_start = fu.get_scene_start(scene_df)
    row_dict['scene_start'] = time_start

    # Iterate through each bleeding patient in the scene
    bleeding_mask_series = scene_df.injury_treated_required_procedure.isin(['tourniquet', 'woundpack'])
    for patient_id, tourniquet_required_patient_df in scene_df[bleeding_mask_series].groupby('patient_id'):
        for cn in fu.patient_groupby_columns: row_dict[cn] = eval(cn)
        row_dict['is_patient_bleeding'] = fu.is_injury_hemorrhage(patient_df)
        
        # Get the entire history of this patient
        mask_series = True
        for cn in fu.scene_groupby_columns: mask_series &= (frvrs_logs_df[cn] == eval(cn))
        mask_series &= (frvrs_logs_df.patient_id == patient_id)
        patient_df = frvrs_logs_df[mask_series]
        columns_list = sorted([
            'injury_treated_required_procedure', 'injury_record_patient_id', 'injury_treated_patient_id', 'tool_applied_patient_id', 'injury_record_id',
            'injury_treated_id', 'tool_applied_sender', 'injury_treated_injury_treated', 'injury_treated_injury_treated_with_wrong_treatment', 'elapsed_time',
            'action_type', 'file_name', 'patient_id', 'injury_id'
        ])
        display(patient_df[columns_list].dropna(axis='columns', how='all').T); raise
        
        # Get the time of patient engagement
        mask_series = patient_df.action_type.isin(['PATIENT_ENGAGED'])
        patient_engaged = fu.get_scene_start(patient_df[mask_series])
        row_dict['patient_engagement_time'] = patient_engaged
        
        # Get the entire history of the patients that had these injuries
        injury_mask_series = True
        for cn in fu.scene_groupby_columns: injury_mask_series &= (frvrs_logs_df[cn] == eval(cn))
        injury_mask_series &= (
            (frvrs_logs_df.injury_record_patient_id == patient_id) |
            (frvrs_logs_df.injury_treated_patient_id == patient_id) |
            (frvrs_logs_df.tool_applied_patient_id == patient_id)
        )
        
        # Add the entire history of these injuries
        injury_ids_list = patient_df[bleeding_mask_series].injury_treated_id.tolist()
        injury_mask_series &= (
            frvrs_logs_df.injury_record_id.isin(injury_ids_list) |
            frvrs_logs_df.injury_treated_id.isin(injury_ids_list) |
            frvrs_logs_df.injury_id.isnull()
        )
        # injury_mask_series &= frvrs_logs_df.action_type.isin(['INJURY_RECORD', 'TOOL_APPLIED', 'INJURY_TREATED'])
        
        injury_records_df = frvrs_logs_df[injury_mask_series]

        # Was the correct tool applied?
        row_dict['is_correct_tool_applied'] = fu.is_correct_bleeding_tool_applied(injury_records_df)
        mask_series = injury_records_df.tool_applied_sender.isin(['AppliedTourniquet', 'AppliedPackingGauze'])
        if injury_records_df[mask_series].shape[0]:
            
            # Get the last tourniquet applied
            time_stop = fu.get_scene_end(injury_records_df[mask_series])
            row_dict['tool_applied_time'] = time_stop
            
            # Get the time elapsed
            row_dict['run_tool_control_time'] = time_stop - time_start
            row_dict['engaged_tool_control_time'] = time_stop - patient_engaged
        
        # Was the bleeding treated?
        is_bleeding_treated = fu.is_injury_correctly_treated(injury_records_df)
        row_dict['is_bleeding_treated'] = is_bleeding_treated
        if is_bleeding_treated:
            
            # Get the last tourniquet applied
            time_stop = fu.get_injury_correctly_treated_time(injury_records_df)
            row_dict['bleeding_treated_time'] = time_stop
            
            # Get the time elapsed
            row_dict['run_bleeding_control_delta'] = time_stop - time_start
            row_dict['engaged_bleeding_control_delta'] = time_stop - patient_engaged
        
        elif (row_dict['is_correct_tool_applied'] == True): 
            columns_list = ['injury_treated_injury_treated', 'tool_applied_sender']
            idx_list = injury_records_df[columns_list].drop_duplicates().index.tolist()
            mask_series = frvrs_logs_df.index.isin(idx_list)
            columns_list = sorted([
                'injury_treated_required_procedure', 'injury_record_patient_id', 'injury_treated_patient_id', 'tool_applied_patient_id', 'injury_record_id',
                'injury_treated_id', 'tool_applied_sender', 'injury_treated_injury_treated', 'injury_treated_injury_treated_with_wrong_treatment', 'elapsed_time',
                'action_type', 'file_name'
            ])
            df = frvrs_logs_df[mask_series][columns_list].sort_values('elapsed_time')
            if (df.tool_applied_sender.unique().shape[0] > 1):
                if is_first_one:
                    display(df.T)
                    is_first_one = False
                file_name = fu.get_file_name(df)
                file_names_set.add(file_name)
                print(
                    f'In {file_name} (or {fu.get_new_file_name(file_name)}), search for these integers in the second column:'
                    f"\n(?:^[^,]+,)({'|'.join([str(n) for n in df.elapsed_time.unique()])})(?:,)\n"
                )
    
    rows_list.append(row_dict)
run_control_time_df = DataFrame(rows_list)

,6,8,10,12,241,242,244,246,247,251,...,827490,828157,828158,828846,828848,829001,829002,829003,829004,829005
action_type,INJURY_RECORD,INJURY_RECORD,INJURY_RECORD,INJURY_RECORD,INJURY_RECORD,INJURY_RECORD,INJURY_RECORD,INJURY_RECORD,INJURY_RECORD,INJURY_RECORD,...,INJURY_RECORD,INJURY_RECORD,INJURY_RECORD,INJURY_RECORD,INJURY_RECORD,INJURY_RECORD,INJURY_RECORD,INJURY_RECORD,INJURY_RECORD,INJURY_RECORD
elapsed_time,11992,11992,11992,11992,335549,335549,335549,335549,335549,335549,...,15691,12291,12291,107072,107072,27833,27833,27833,27833,27833
event_time,2023-03-10 07:53:47,2023-03-10 07:53:47,2023-03-10 07:53:47,2023-03-10 07:53:47,2023-03-10 07:59:11,2023-03-10 07:59:11,2023-03-10 07:59:11,2023-03-10 07:59:11,2023-03-10 07:59:11,2023-03-10 07:59:11,...,2023-09-07 15:33:27,2023-09-07 14:49:05,2023-09-07 14:49:05,2023-09-07 08:55:46,2023-09-07 08:55:46,2023-09-07 08:54:27,2023-09-07 08:54:27,2023-09-07 08:54:27,2023-09-07 08:54:27,2023-09-07 08:54:27
session_uuid,cf714cd2-2824-4ad2-8d98-3199b1df0bdb,cf714cd2-2824-4ad2-8d98-3199b1df0bdb,cf714cd2-2824-4ad2-8d98-3199b1df0bdb,cf714cd2-2824-4ad2-8d98-3199b1df0bdb,cf714cd2-2824-4ad2-8d98-3199b1df0bdb,cf714cd2-2824-4ad2-8d98-3199b1df0bdb,cf714cd2-2824-4ad2-8d98-3199b1df0bdb,cf714cd2-2824-4ad2-8d98-3199b1df0bdb,cf714cd2-2824-4ad2-8d98-3199b1df0bdb,cf714cd2-2824-4ad2-8d98-3199b1df0bdb,...,f8b5cbd1-0abe-483b-8b16-956044dc0119,fde2c201-36fc-4123-bf42-f1bff1b24969,fde2c201-36fc-4123-bf42-f1bff1b24969,ff4c3532-5924-45b6-9a6a-d0d3274b6ef0,ff4c3532-5924-45b6-9a6a-d0d3274b6ef0,ff4c3532-5924-45b6-9a6a-d0d3274b6ef0,ff4c3532-5924-45b6-9a6a-d0d3274b6ef0,ff4c3532-5924-45b6-9a6a-d0d3274b6ef0,ff4c3532-5924-45b6-9a6a-d0d3274b6ef0,ff4c3532-5924-45b6-9a6a-d0d3274b6ef0
file_name,All CSV files renamed by date/03.10.23.0753.csv,All CSV files renamed by date/03.10.23.0753.csv,All CSV files renamed by date/03.10.23.0753.csv,All CSV files renamed by date/03.10.23.0753.csv,All CSV files renamed by date/03.10.23.0753.csv,All CSV files renamed by date/03.10.23.0753.csv,All CSV files renamed by date/03.10.23.0753.csv,All CSV files renamed by date/03.10.23.0753.csv,All CSV files renamed by date/03.10.23.0753.csv,All CSV files renamed by date/03.10.23.0753.csv,...,v.1.3/f8b5cbd1-0abe-483b-8b16-956044dc0119.csv,v.1.3/fde2c201-36fc-4123-bf42-f1bff1b24969.csv,v.1.3/fde2c201-36fc-4123-bf42-f1bff1b24969.csv,v.1.3/ff4c3532-5924-45b6-9a6a-d0d3274b6ef0.csv,v.1.3/ff4c3532-5924-45b6-9a6a-d0d3274b6ef0.csv,v.1.3/ff4c3532-5924-45b6-9a6a-d0d3274b6ef0.csv,v.1.3/ff4c3532-5924-45b6-9a6a-d0d3274b6ef0.csv,v.1.3/ff4c3532-5924-45b6-9a6a-d0d3274b6ef0.csv,v.1.3/ff4c3532-5924-45b6-9a6a-d0d3274b6ef0.csv,v.1.3/ff4c3532-5924-45b6-9a6a-d0d3274b6ef0.csv
logger_version,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.3,1.3,1.3,1.3,1.3,1.3,1.3,1.3,1.3,1.3
injury_record_id,R Chest Collapse,L Shoulder Puncture,L Thigh Laceration,R Forearm Laceration,R Side Puncture,R Thigh Laceration,R Calf Shrapnel,L Thigh Puncture,L Forearm Laceration,Ear Bleed,...,L Neck Puncture,L Neck Puncture,R Shin Amputation,R Shin Amputation,L Neck Puncture,R Forearm Laceration,L Bicep Puncture,L Shoulder Puncture,R Side Puncture,L Thigh Laceration
injury_record_patient_id,Mike_3 Root,Mike_2 Root,Mike_1 Root,Mike_0 Root,Mike_5 Root,Mike_5 Root,Gloria_6 Root,Mike_7 Root,Gloria_8 Root,Gary_9 Root,...,Military Mike Jungle Burned_0 Root,Military Mike Jungle Burned_0 Root,Military Mike Jungle Combat_3_2 Root,Military Mike Jungle Combat_3_2 Root,Military Mike Jungle Burned_0 Root,Military Mike Jungle Combat_2_0 Root,Military Mike Jungle Combat_2_0 Root,Military Mike Jungle Combat_2_0 Root,Military Mike Jungle Combat_2_0 Root,Military Mike Jungle Combat_2_0 Root
injury_record_required_procedure,decompress,woundpack,tourniquet,gauzePressure,woundpack,tourniquet,gauzePressure,tourniquet,gauzePressure,none,...,woundpack,woundpack,tourniquet,tourniquet,woundpack,gauzePressure,tourniquet,woundpack,woundpack,tourniquet
injury_record_severity,high,medium,medium,low,medium,medium,medium,m

RuntimeError: No active exception to reraise

In [13]:

set([
    'injury_treated_required_procedure', 'injury_record_patient_id', 'injury_treated_patient_id', 'tool_applied_patient_id', 'injury_record_id',
    'injury_treated_id', 'tool_applied_sender', 'injury_treated_injury_treated', 'injury_treated_injury_treated_with_wrong_treatment', 'elapsed_time',
    'action_type', 'file_name'
]) - set([
    'action_type', 'elapsed_time', 'event_time', 'session_uuid', 'file_name', 'logger_version', 'patient_demoted_health_level', 'patient_demoted_health_time_remaining',
    'patient_demoted_id', 'patient_demoted_position', 'patient_demoted_rotation', 'patient_demoted_salt', 'patient_demoted_sort', 'patient_demoted_pulse',
    'patient_demoted_breath', 'patient_demoted_hearing', 'patient_demoted_mood', 'patient_demoted_pose', 'patient_record_health_level',
    'patient_record_health_time_remaining', 'patient_record_id', 'patient_record_position', 'patient_record_rotation', 'patient_record_salt', 'patient_record_sort',
    'patient_record_pulse', 'patient_record_breath', 'patient_record_hearing', 'patient_record_mood', 'patient_record_pose', 'injury_record_id',
    'injury_record_patient_id', 'injury_record_required_procedure', 'injury_record_severity', 'injury_record_body_region', 'injury_record_injury_treated',
    'injury_record_injury_treated_with_wrong_treatment', 'injury_record_injury_injury_locator', 's_a_l_t_walk_if_can_sort_location',
    's_a_l_t_walk_if_can_sort_command_text', 's_a_l_t_walk_if_can_patient_id', 's_a_l_t_wave_if_can_sort_location', 's_a_l_t_wave_if_can_sort_command_text',
    's_a_l_t_wave_if_can_patient_id', 's_a_l_t_waved_sort_location', 's_a_l_t_waved_sort_command_text', 's_a_l_t_waved_patient_id', 'pulse_taken_pulse_name',
    'pulse_taken_patient_id', 'patient_engaged_health_level', 'patient_engaged_health_time_remaining', 'patient_engaged_id', 'patient_engaged_position',
    'patient_engaged_rotation', 'patient_engaged_salt', 'patient_engaged_sort', 'patient_engaged_pulse', 'patient_engaged_breath', 'patient_engaged_hearing',
    'patient_engaged_mood', 'patient_engaged_pose', 'injury_treated_id', 'injury_treated_patient_id', 'injury_treated_required_procedure', 'injury_treated_severity',
    'injury_treated_body_region', 'injury_treated_injury_treated', 'injury_treated_injury_treated_with_wrong_treatment', 'injury_treated_injury_injury_locator',
    'tool_applied_type', 'tool_applied_attachment_point', 'tool_applied_tool_location', 'tool_applied_data', 'tool_applied_sender', 'tag_applied_patient_id',
    'tag_applied_type', 'tool_applied_patient_id', 'scene_index', 'patient_id', 'location_id', 'injury_id', 'is_scene_aborted', 'scene_type', 'is_a_one_triage_file'
])

set()


## Action

Ashu called a meeting with Brian Pippin and others as this fork of the FRVRS logger isn't the only one. Dave is making this notebook to demonstrate the problem.